# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6hSleeping whole brain DBA/2J strain (D2) Expression values and Group 2 be 6hSD whole brain DBA/2J strain (D2) Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_ZT6_D2_'   # define a prefix to add to column names (mD2ing indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9442.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 87)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 87)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='D2_S_ZT6.CEL'
sd_filt ='D2_SD_ZT6.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239915_D2_S_ZT6.CEL,GSM239916_D2_S_ZT6.CEL,GSM239917_D2_S_ZT6.CEL
Probesets,,,
1427138_at,6.01902,6.09050,6.15032
1425600_a_at,7.55049,7.66302,7.63895
1457168_at,4.71146,4.67901,4.60366
1450135_at,4.98641,5.03533,4.98962
1424014_at,8.31202,8.12028,8.04793


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239927_D2_SD_ZT6.CEL,GSM239928_D2_SD_ZT6.CEL,GSM239929_D2_SD_ZT6.CEL
Probesets,,,
1427138_at,6.01677,6.11602,6.20855
1425600_a_at,7.48872,7.48146,7.39999
1457168_at,4.58878,4.70513,4.49720
1450135_at,5.06413,4.97521,5.11886
1424014_at,8.23946,8.18405,8.23689


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.027167
1425600_a_at   -0.160763
1457168_at     -0.067673
1450135_at      0.048947
1424014_at      0.060057
Name: 430AV2_ZT6_D2_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at      0.330419
1425600_a_at   -2.950298
1457168_at     -0.811218
1450135_at      0.893263
1424014_at      0.606635
Name: 430AV2_ZT6_D2_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      0.280856
1425600_a_at   -2.507754
1457168_at     -0.689535
1450135_at      0.759274
1424014_at      0.515640
Name: 430AV2_ZT6_D2_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.675765
1425600_a_at    1.392022
1457168_at      0.721506
1450135_at      0.733160
1424014_at      0.697334
Name: 430AV2_ZT6_D2_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_ZT6_D2_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_ZT6_D2_Enrich,430AV2_ZT6_D2_poolStDev,430AV2_ZT6_D2_Cohens_d,430AV2_ZT6_D2_J,430AV2_ZT6_D2_Hedges_g,430AV2_ZT6_D2_Var_d,430AV2_ZT6_D2_Var_g,430AV2_ZT6_D2_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1445343_at,Tmed5,transmembrane emp24 protein transport domain c...,ENSMUSG00000063406,73130,NaN,transport,NaN,endoplasmic reticulum membrane // endoplasmic ...,NaN,"mmu-let-7(TargetScan), mmu-let-7a(miRanda|pict...",...,4.759563e-05,0.289457,0.244213,0.005699,42.851688,0.85,36.423935,153.688931,111.040253,10.537564
1416480_a_at,ENSMUSG00000044330 /// Higd1a /// LOC100045763,"HIG1 domain family, member 1A",ENSMUSG00000044330|ENSMUSG00000038412,100045763|56295|100042265,NaN,response to stress,NaN,integral to membrane // membrane,NaN,"mmu-miR-103(pictar), mmu-miR-107(pictar), mmu-...",...,2.262908e-04,0.354053,0.192127,0.010212,18.814520,0.85,15.992342,30.165513,21.794583,4.668467
1439438_a_at,1110005A23Rik /// EG625193,SAP domain containing ribonucleoprotein,ENSMUSG00000051255|ENSMUSG00000078427,625193|66118,NaN,"transcription // regulation of transcription, ...",nucleic acid binding // protein binding // DNA...,nucleus,NaN,NaN,...,3.069488e-04,0.257413,0.237917,0.013539,17.573074,0.85,14.937113,26.401078,19.074779,4.367468
1422676_at,Smarce1,"SWI/SNF related, matrix associated, actin depe...",ENSMUSG00000037935,57376,NaN,chromatin modification,DNA binding,nucleus,TNF-alpha NF-kB Signaling Pathway:WP246(WikiPa...,"mmu-miR-103a(TargetScan), mmu-miR-107(TargetSc...",...,2.976215e-07,0.481577,0.152733,0.010971,13.920985,0.85,11.832838,16.816153,12.149671,3.485638
1428136_at,Sfrp1,secreted frizzled-related protein 1 [Source:MG...,ENSMUSG00000031548,20377,NaN,anterior/posterior pattern formation // multic...,protein binding,extracellular region,BMP Signaling Pathway in Eyelid Development:WP...,"mmu-miR-1(miRanda|pictar), mmu-miR-124(miRanda...",...,6.246915e-01,0.328150,0.162397,0.014480,11.215052,0.85,9.532794,11.148115,8.054513,2.838047
1418350_at,Hbegf,heparin-binding EGF-like growth factor,ENSMUSG00000024486,15200,NaN,regulation of heart contraction // positive re...,epidermal growth factor receptor binding // gr...,extracellular space // extracellular region //...,ErbB signaling pathway:WP1261(WikiPathways) //...,"mmu-let-7c(miRanda), mmu-miR-1192(TargetScan|m...",...,7.551282e-04,0.433440,0.218943,0.019748,11.086844,0.85,9.423817,10.909842,7.882361,2.807554
1428484_at,Osbpl3,oxysterol binding protein-like 3 [Source:MGI S...,ENSMUSG00000029822,71720,NaN,lipid transport // steroid metabolic process /...,NaN,NaN,NaN,"mmu-let-7(TargetScan), mmu-let-7a(pictar), mmu...",...,5.428254e-07,0.632187,0.264150,0.025526,10.348433,0.85,8.796168,9.590839,6.929381,2.632372
1424244_at,Rwdd4a,RWD domain containing 4A,ENSMUSG00000031568,192174,NaN,NaN,NaN,NaN,NaN,"mmu-miR-103(miRanda), mmu-miR-107(miRanda), mm...",...,1.768311e-01,0.264650,0.228903,0.022244,10.290553,0.85,8.746970,9.491291,6.857458,2.618675
1451822_a_at,Scrn2,secernin 2,ENSMUSG00000020877,217140,NaN,proteolysis,dipeptidase activity,NaN,NaN,"mmu-miR-224(miRanda), mmu-miR-490(miRanda), mm...",...,1.604936e-03,0.395303,0.395303,0.040208,9.831465,0.85,8.356745,8.721475,6.301265,2.510232


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL,avg-AK_S_ZT6,log_fold-AK_S_ZT6_vs_AK_SD_ZT6,fold-AK_S_ZT6_vs_AK_SD_ZT6,rawp-AK_S_ZT6_vs_AK_SD_ZT6,adjp-AK_S_ZT6_vs_AK_SD_ZT6,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,...,ANOVA-adjp,largest fold,430AV2_ZT6_D2_Enrich,430AV2_ZT6_D2_poolStDev,430AV2_ZT6_D2_Cohens_d,430AV2_ZT6_D2_J,430AV2_ZT6_D2_Hedges_g,430AV2_ZT6_D2_Var_d,430AV2_ZT6_D2_Var_g,430AV2_ZT6_D2_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_ZT6_D2_Enrich',u'430AV2_ZT6_D2_Hedges_g', u'430AV2_ZT6_D2_Var_g', u'430AV2_ZT6_D2_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_ZT6_D2_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_ZT6_D2_Enrich,430AV2_ZT6_D2_Hedges_g,430AV2_ZT6_D2_Var_g,430AV2_ZT6_D2_SEg
MGI symbol,,,,
0610005C13Rik,0.089680,0.902413,0.549529,0.741302
0610008F07Rik,-0.040547,-0.177167,0.484282,0.695904
0610009B22Rik,0.062817,0.616777,0.513368,0.716497
0610009D07Rik,0.146680,1.714562,0.768442,0.868837
0610009O20Rik,-0.143390,-1.760885,0.740060,0.860267
0610010K14Rik,0.096367,0.434760,0.497418,0.705279
0610012G03Rik,0.130631,1.059977,0.592152,0.767803
0610031J06Rik,0.017660,0.109815,0.482672,0.694746
0610037L13Rik,0.092313,0.769596,0.531023,0.728713
